In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ========= 1. Load combined signals =========
signals_db = Path("data") / "signals" / "combined.sqlite"

with sqlite3.connect(signals_db) as conn:
    df = pd.read_sql("SELECT * FROM signals_combined", conn, parse_dates=["Date"])

df = df.set_index("Date").sort_index()

# ========= 2. Ensure required columns =========
if "Close" not in df.columns:
    raise KeyError("'Close' column not found in signals_combined table")
if "signal_combined" not in df.columns:
    raise KeyError("'signal_combined' column not found in signals_combined table")

# ========= 3. Vectorized backtest =========
df["returns"] = df["Close"].pct_change().fillna(0)

# Strategy returns: shift signal by 1 to avoid look-ahead bias
df["strategy_returns"] = df["signal_combined"].shift(1).fillna(0) * df["returns"]

# Cumulative equity curves (start at 1.0)
df["buy_hold_eq"] = (1 + df["returns"]).cumprod()
df["strategy_eq"] = (1 + df["strategy_returns"]).cumprod()

# ========= 4. Performance metrics =========
total_return_bh  = df["buy_hold_eq"].iloc[-1] - 1
total_return_strat = df["strategy_eq"].iloc[-1] - 1

sharpe_bh = df["returns"].mean() / df["returns"].std() * np.sqrt(252) if df["returns"].std() != 0 else 0
sharpe_strat = df["strategy_returns"].mean() / df["strategy_returns"].std() * np.sqrt(252) if df["strategy_returns"].std() != 0 else 0

# Max drawdown
def max_drawdown(equity_curve):
    running_max = equity_curve.cummax()
    drawdown = (equity_curve - running_max) / running_max
    return drawdown.min()

mdd_bh = max_drawdown(df["buy_hold_eq"])
mdd_strat = max_drawdown(df["strategy_eq"])

print("========== Backtest Results ==========")
print(f"Period: {df.index[0]} to {df.index[-1]}")
print(f"Total days: {len(df)}")
print()
print(f"Buy & Hold Total Return: {total_return_bh:.2%}")
print(f"Strategy Total Return:   {total_return_strat:.2%}")
print()
print(f"Buy & Hold Sharpe Ratio: {sharpe_bh:.2f}")
print(f"Strategy Sharpe Ratio:   {sharpe_strat:.2f}")
print()
print(f"Buy & Hold Max Drawdown: {mdd_bh:.2%}")
print(f"Strategy Max Drawdown:   {mdd_strat:.2%}")
print("======================================")

# ========= 5. Plot equity curves =========
plt.figure(figsize=(12, 6))
plt.plot(df.index, df["buy_hold_eq"], label="Buy & Hold", color="blue", linewidth=1.5)
plt.plot(df.index, df["strategy_eq"], label="Combined Signal Strategy", color="green", linewidth=1.5)
plt.title("Vectorized Backtest: Equity Curves")
plt.xlabel("Date")
plt.ylabel("Equity (starting at 1.0)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ========= 6. Save backtest results =========
backtest_db = Path("data") / "backtest" / "backtest_results.sqlite"
backtest_db.parent.mkdir(parents=True, exist_ok=True)

with sqlite3.connect(backtest_db) as conn:
    df.reset_index().to_sql("backtest", conn, if_exists="replace", index=False)

print("Saved backtest results to:", backtest_db.resolve())